In [1]:
!pip install tensorflow-gpu keras tensorflow-hub

     |████████████████████████████████| 421.8MB 37kB/s 
     |████████████████████████████████| 450kB 38.9MB/s 
     |████████████████████████████████| 3.9MB 39.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=43cf5ee07c5b5c4889895e75b26fb19b5d83b8226fea715b3357d3a141532797
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0rc3 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0rc3 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 2.1.1 which is incompatible.
ERROR: tensorflow 2.2.0rc3 has requirement tensorflow-estimator<2.3.0,>=2.2.0rc0, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Fo

In [2]:
import tensorflow as tf
import pandas as pd
import os
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import matplotlib.style as style
from datetime import datetime
from keras.preprocessing import image
from PIL import Image
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve
from tensorflow.keras import layers
from time import time
print("TF version:", tf.__version__)

Using TensorFlow backend.


TF version: 2.1.0


In [0]:
zip_file=tf.keras.utils.get_file(origin=' http://03fe90ee.ngrok.io/dataset.zip', 
  fname='dataset.zip', extract=True)
train_dir = os.path.join(os.path.dirname(zip_file),'new_train')
validation_dir = os.path.join(os.path.dirname(zip_file),'new_test')

 997203968/1810474624 [===============>..............] - ETA: 24:26

In [0]:
IMG_SIZE = 224  
CHANNELS = 3

In [0]:
ls /root/.keras/datasets/

In [0]:
def parse_function(filename, label):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=CHANNELS)
    image_resized = tf.image.resize(image_decoded, [IMG_SIZE, IMG_SIZE])
    image_normalized = image_resized / 255.0
    return image_normalized, label

In [0]:
BATCH_SIZE = 256
AUTOTUNE = tf.data.experimental.AUTOTUNE
SHUFFLE_BUFFER_SIZE = 1024

In [0]:
def create_dataset(filenames, labels, is_training=True):
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    dataset = dataset.map(parse_function, num_parallel_calls=AUTOTUNE)
    if is_training == True:
        dataset = dataset.cache()
        dataset = dataset.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE)
    
        dataset = dataset.batch(BATCH_SIZE)
        dataset = dataset.prefetch(buffer_size=AUTOTUNE)

        return dataset


In [0]:
train = pd.read_csv('new_train.csv')
train.category = train.category.apply(lambda s: [l for l in str(s).split(',')])
print(train.head())
train_files = [os.path.join(train_dir, file) for file in train.image]
train_labels = list(train.category)

In [0]:
test = pd.read_csv('new_test.csv')
test.category = test.category.apply(lambda s: [l for l in str(s).split(',')])
print(test.head())
test_files = [os.path.join(validation_dir, file) for file in test.image]
test_labels = list(test.category)

In [0]:
xtrain, xtest, ytrain, ytest = train_files, test_files, train_labels, test_labels

In [0]:
nobs = 8 # Maximum number of images to display
ncols = 4 # Number of columns in display
nrows = nobs//ncols # Number of rows in display

style.use("default")
plt.figure(figsize=(12,4*nrows))
for i in range(nrows*ncols):
    ax = plt.subplot(nrows, ncols, i+1)
    plt.imshow(Image.open(xtrain[i]))
    plt.title(ytrain[i], size=10)
    plt.axis('off')

In [0]:
mlb = MultiLabelBinarizer()
mlb.fit(ytrain)
N_LABELS = len(mlb.classes_)
for (i, label) in enumerate(mlb.classes_):
    print("{}. {}".format(i, label))

In [0]:
ytrainbin = mlb.transform(ytrain)
ytestbin = mlb.transform(ytest)

In [0]:
for i in range(3):
    print(xtrain[i], ytrainbin[i])

In [0]:
train = create_dataset(xtrain, ytrainbin)
test = create_dataset(xtest, ytestbin)

In [0]:
for f, l in train.take(1):
    print("Shape of features array:", f.numpy().shape)
    print("Shape of labels array:", l.numpy().shape)

In [0]:
feature_extractor_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4"
feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                         input_shape=(IMG_SIZE,IMG_SIZE,CHANNELS))

In [0]:
feature_extractor_layer.trainable = False

In [0]:
model = tf.keras.Sequential([
    feature_extractor_layer,
    layers.Dense(1024, activation='relu', name='hidden_layer'),
    layers.Dense(N_LABELS, activation='sigmoid', name='output')
])

model.summary()

In [0]:
for batch in train:
    print(model.predict(batch)[:1])
    break

In [0]:
@tf.function
def macro_soft_f1(y, y_hat):

    y = tf.cast(y, tf.float32)
    y_hat = tf.cast(y_hat, tf.float32)
    tp = tf.reduce_sum(y_hat * y, axis=0)
    fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
    soft_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    cost = 1 - soft_f1 # reduce 1 - soft-f1 in order to increase soft-f1
    macro_cost = tf.reduce_mean(cost) # average on all labels
    return macro_cost

In [0]:
@tf.function
def macro_f1(y, y_hat, thresh=0.5):
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

In [0]:
LR = 1e-5 # Keep it small when transfer learning
EPOCHS = 30

In [0]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
  loss=macro_soft_f1,
  metrics=[macro_f1])

In [0]:
start = time()
history = model.fit(train,
                    epochs=EPOCHS,
                    validation_data=create_dataset(xtest, ytestbin))
print('\nTraining took {}'.format(print_time(time()-start)))

In [0]:
model_bce = tf.keras.Sequential([
    feature_extractor_layer,
    layers.Dense(N_LABELS, activation='sigmoid')
])

model_bce.compile(
    optimizer=tf.keras.optimizers.Adam(lr=5e-4),
    loss=tf.keras.metrics.binary_crossentropy,
    metrics=[macro_f1])
    
start = time()
history_bce = model_bce.fit(train,
                            epochs=EPOCHS,
                            validation_data=create_dataset(xtest, ytestbin))
print('\nTraining took {}'.format(print_time(time()-start)))